In [3]:
%cd ..

/home/ead/iking5/code/bitcoin_fraud_detection


In [17]:
import torch 
from models.inductive_graph_gan import GeneratorClassifier
from inductive_graphgan import prepare_btc_data

g = torch.load('resources/btc_graph_unsplit.pt')
_,gsd, hidden, latent = torch.load('saved_models/inductive_gan_cls.pt')
gen = GeneratorClassifier(g.x.size(1)-1, hidden, latent)
gen.load_state_dict(gsd)
gen.eval()

GeneratorClassifier(
  (first): Sequential(
    (0): Linear(in_features=2, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (mu): Sequential(
    (0): Linear(in_features=4096, out_features=2048, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=2048, out_features=64, bias=True)
  )
  (logvar): Sequential(
    (0): Linear(in_features=4096, out_features=2048, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=2048, out_features=64, bias=True)
  )
  (decode): Sequential(
    (0): Linear(in_features=64, out_features=2048, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=2048, out_features=4096, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=4096, out_features=165, bias=True)
  )
)

In [15]:
tr_x = g.x[g.x[:,0] < 35][:,1:]
tr_y = g.y[g.x[:,0] < 35]

is_unk = tr_y == 0 
tr_y = tr_y[~is_unk]
tr_x = tr_x[~is_unk]
tr_y[tr_y == 2] = 0

te_x = g.x[g.x[:,0] >= 35][:,1:]
te_y = g.y[g.x[:,0] >= 35]

is_unk = te_y == 0 
te_y = te_y[~is_unk]
te_x = te_x[~is_unk]
te_y[te_y == 2] = 0 

In [18]:
from sklearn.ensemble import RandomForestClassifier as RF 
from sklearn.metrics import (
    accuracy_score, precision_score,
    recall_score, f1_score
)

def cls(tr_x, tr_y, te_x, te_y):
    rf = RF(n_jobs=16)
    rf.fit(tr_x, tr_y)
    preds = rf.predict(te_x)

    print("Accuracy: ", accuracy_score(te_y, preds))
    print("Precision: ", precision_score(te_y, preds))
    print("Recall: ", recall_score(te_y, preds))
    print("F1: ", f1_score(te_y, preds))

print("Default:")
cls(tr_x, tr_y, te_x, te_y)

# Generate synthetic negative samples
num_zeros = (tr_y == 0).sum()
input = torch.zeros(num_zeros, 2)
input[:,1] = 1. 

gen.eval()
with torch.no_grad():
    enhanced = gen.sample(input, variational=True)

labels = torch.ones(enhanced.size(0))

print("\nSynthetic negative samples:")
cls(torch.cat([tr_x, enhanced]), torch.cat([tr_y, labels]), te_x, te_y)

Default:
Accuracy:  0.9794841031793641
Precision:  0.9501822600243013
Recall:  0.7220683287165282
F1:  0.8205666316894018

Synthetic negative samples:
Accuracy:  0.9803239352129574
Precision:  0.9677819083023544
Recall:  0.7211449676823638
F1:  0.8264550264550264
